In [30]:
import pandas as pd

from ipymarkup import show_span_ascii_markup as show_markup

from yargy import rule, Parser, or_
from yargy.predicates import eq, type as type_, gram, normalized
from yargy.pipelines import morph_pipeline, caseless_pipeline
from yargy.interpretation import fact

In [31]:
df = pd.read_csv('../test_data.csv')
df.text = df.text.str.lower()

Нам необходимы реплики приветствия и прощания, поэтому будем рассматривать первые и последние 5 реплик менеджера

In [32]:
# функция для выбора необходимых строк
def cut_manager_part(df, number_of_dialogue, border = 5):
    dig_num = number_of_dialogue

    dlg_df_manager = df[df['role'] == 'manager']
    dlg_df_manager = dlg_df_manager[dlg_df_manager['dlg_id'] == dig_num]
    dlg_df_manager_cut = pd.concat([dlg_df_manager.iloc[:border],  dlg_df_manager.iloc[-border:]])
    return dlg_df_manager_cut['text'].to_list()

In [33]:
# функция для применения правил описанных с помощью библитеки yargy
# к искомому датафрейму. Функция возвращает словарь с парой идентификатор диалога
# и выделенной по правилу фразой
def df_processor(df, rule, print_text = True):
    loc_dict = {}
    parser = Parser(rule)
    for i in range(len(df['dlg_id'].value_counts())):
        if print_text:
            print("Number of dialogue # " + str(i))
        dlg_list = cut_manager_part(df, i)
        for line in dlg_list:
            matches = list(parser.findall(line))
            if print_text:
                spans = [_.span for _ in matches]
                show_markup(line, spans)
            if matches:            
                for match in parser.findall(line):
                    loc_dict[i] = [_.value for _ in match.tokens]
        if print_text:
            print('\n')
            
    return loc_dict

# Выделение приветстивия

In [34]:
greeting_dict = {}

GREETING = morph_pipeline([
    'добрый день',
    'добрый вечер',
    'доброе утро',
    'доброго времени',
    'здравствуйте'
])

greeting_dict = df_processor(df, GREETING, print_text=True)

greeting_dict

Number of dialogue # 0
алло здравствуйте
     ────────────
меня зовут ангелина компания диджитал бизнес звоним вам по поводу 
продления лицензии а мы с серым у вас скоро срок заканчивается
угу ну возможно вы рассмотрите и другие варианты видите это хорошая 
практика сравнивать
угу а на что вы обращаете внимание при выборе
что для вас приоритет
вот понимаю да но дело в том что в основном да за месяц
необходимо обзванивать
хорошо я данную информацию тогда зафиксирую передам будем иметь ввиду
и вам спасибо большое за обратную связь
всего хорошего до свидания


Number of dialogue # 1
алло здравствуйте
     ────────────
меня зовут ангелина компания диджитал бизнес звоню вам по поводу 
продления а мы сели обратила внимание что у вас срок заканчивается
а так у вас заканчивается 25 февраля
на полгода минимальный
а у вас сейчас 8 базовый правильно
а куда вам можно на ватсап например
давайте тогда почту проверим инфо собачка эра черточка 3 дэ точка ру 
правильно
угу
угу да вижу я эту почту хорош

{0: ['здравствуйте'],
 1: ['здравствуйте'],
 2: ['здравствуйте'],
 3: ['добрый', 'день']}

# Выделение прощания

In [35]:
farewell_dict = {}

FAREWELL_1 = morph_pipeline([
    'до свидания',
    'всего доброго',
])
FAREWELL_2 = caseless_pipeline([
    'всего хорошего'
])

farewell_dict = df_processor(df, or_(FAREWELL_1, FAREWELL_2), print_text=False)
farewell_dict

{0: ['до', 'свидания'],
 1: ['до', 'свидания'],
 3: ['всего', 'доброго'],
 4: ['до', 'свидания'],
 5: ['до', 'свидания']}

# Выделение представления менеджера

In [36]:
intro_dict = {}

NAME = gram('Name')

INTRO_1 = rule(
    NAME.optional(),
    eq('меня'),
    NAME.optional(),
    normalized('зовут'),
    NAME.optional()
)

INTRO_2 = rule(
    normalized('это'),
    NAME
)

intro_dict = df_processor(df, or_(INTRO_1, INTRO_2), print_text=False)
intro_dict

{0: ['меня', 'зовут', 'ангелина'],
 1: ['меня', 'зовут', 'ангелина'],
 2: ['меня', 'зовут', 'ангелина'],
 3: ['меня', 'максим', 'зовут'],
 5: ['это', 'анастасия']}

# Выделение имени менеджера

In [37]:
name_dict = {}
parser = Parser(rule(NAME))
for i, j in zip(intro_dict.keys(), intro_dict.values()):
    line = ' '.join(j)
    matches = list(parser.findall(line))
    if matches: 
        for match in parser.findall(line):
            name_dict[i] = ([_.value for _ in match.tokens])
name_dict

{0: ['ангелина'],
 1: ['ангелина'],
 2: ['ангелина'],
 3: ['максим'],
 5: ['анастасия']}

# Выделение названия компании

In [38]:
companyname_dict = {}

NAME = gram('Name')
NOUN = gram('NOUN')
COMPANY = rule(
        normalized('компания'),
        NOUN
)
companyname_dict = df_processor(df, COMPANY, print_text=False)
companyname_dict

{0: ['компания', 'диджитал'],
 1: ['компания', 'диджитал'],
 2: ['компания', 'диджитал'],
 3: ['компания', 'китобизнес']}

# Объединим результаты в один датафрейм

In [39]:
def dicts_to_df(proc_dict, column_name):
    value_list = []
    for i in list(proc_dict.values()):
        value_list.append(' '.join(i))
    key_list = list(proc_dict.keys())
    result_df = pd.DataFrame(value_list, key_list, columns=[column_name])
    return result_df.reset_index()

In [40]:
dict_list = [name_dict, companyname_dict, intro_dict, greeting_dict, farewell_dict]
columns_name = ['manager_name', 'company_name', 'intro', 'greeting', 'farewell']
df_result = pd.DataFrame({'index' : df.dlg_id.unique()})

for i,j in zip(dict_list, columns_name):
    df_cur = dicts_to_df(i, j)
    df_result = df_result.merge(df_cur, how='outer')

df_result = df_result.rename({'index':'dlg_id'}, axis = 1)

In [41]:
df_result['requirements'] = True
df_result['requirements'][(df_result['greeting'].isna()) | (df_result['farewell'].isna())] = False

C:\Users\xaker_kuznechik\AppData\Local\Temp\ipykernel_4764\2630670035.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result['requirements'][(df_result['greeting'].isna()) | (df_result['farewell'].isna())] = False


In [42]:
df_result

,dlg_id,manager_name,company_name,intro,greeting,farewell,requirements
0,0,ангелина,компания диджитал,меня зовут ангелина,здравствуйте,до свидания,True
1,1,ангелина,компания диджитал,меня зовут ангелина,здравствуйте,до свидания,True
2,2,ангелина,компания диджитал,меня зовут ангелина,здравствуйте,NaN,False
3,3,максим,компания китобизнес,меня максим зовут,добрый день,всего доброго,True
4,4,NaN,NaN,NaN,NaN,до свидания,False
5,5,анастасия,NaN,это анастасия,NaN,до свидания,False
